In [1]:
import os

In [2]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\Loan-Amount-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop\\ml\\Loan-Amount-Prediction'

In [5]:
import dagshub
import mlflow

dagshub.init(repo_owner='richardmukechiwa', repo_name='Loan-Amount-Prediction', mlflow=True)

with mlflow.start_run():
    mlflow.log_param('model_type', 'Linear Regression')
    mlflow.log_metric('rmse', 0.25)

print("MLflow tracking initialized successfully!")

Accessing as richardmukechiwa

Initialized MLflow to track repo "richardmukechiwa/Loan-Amount-Prediction"

Repository richardmukechiwa/Loan-Amount-Prediction initialized!

MLflow tracking initialized successfully!


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name:Path
    target_column: str
    mlflow_uri:str


In [7]:
from credit_risk.constants import *
from credit_risk.utils.common import read_yaml, create_directories, save_json

In [10]:
#creating a Configuration class
class ConfigurationManager:
    def __init__(
        self,
        config_filepath   = CONFIG_FILE_PATH,
        params_filepath   = PARAMS_FILE_PATH,
        schema_filepath   = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)                       
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config= self.config.model_evaluation
        params= self.params.LinearRegression
        schema= self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config=ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            
        )
        
        return model_evaluation_config
        

In [ ]:
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


In [11]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config=config
        
    def eval_metrics(self, actual, pred):
        rmse=np.sqrt(mean_squared_error(actual, pred))
        mae=mean_absolute_error(actual, pred)
        r2= r2_score(actual, pred)
        return rmse, mae, r2
        
        
        